# Panel Regression - Review Variance

In [4]:
# global imports
from toolbox import ToolBox
from tqdm import tqdm
from models import *
import pandas as pd
import numpy as np
import datetime
from dateutil.relativedelta import relativedelta
from datetime import date
import math
import matplotlib.pyplot as plt
import seaborn as sns
from linearmodels.panel import PanelOLS
import statsmodels.api as sm

tqdm.pandas()

## Loading the dataset from the database

In [5]:
tb = ToolBox()

In [6]:
df = tb.load_data_sql(table='user_reviews_clean', use_cache=True)
users = df[~df['review'].isnull()]
users.head()

Loaded from cache


,id,date,grade,game,review,sentiment
0,128002,2013-01-19,30,007-legends-pc,"Oh, my dear God! What is it with developers th...",-0.9191
1,128003,2014-06-23,30,007-legends-pc,Seriously if you want to have a HUGE laugh wit...,0.9424
2,128004,2014-12-11,0,007-legends-pc,godawful port of the xbox 360 version and ps3....,-0.3869
3,128005,2012-11-03,0,007-legends-pc,Agree. Worst game ever. Its a full copy of Cal...,0.5496
4,128006,2012-11-04,0,007-legends-pc,DO NOT BUY THIS GAME for the PC (or for any ot...,-0.9612


## Data cleaning and preprocessing
### User preprocessing

In [7]:
def user_preprocessing(df, timeframe, n_periods):
    
    # getting the minimum review data of a game as proxy for release date
    min_dates = pd.DataFrame(df.groupby('game', as_index=False).agg({'date': 'min'}))
    min_dates.rename(columns={'date':'release_date'}, inplace=True)
    
    # merge release date with df
    preprocessed_df = pd.merge(df, min_dates, on='game')
    
    # define a function to calculate difference in weeks between release date and review date
    def difference_weeks(date, release_date):
        date_dif=date-release_date
        weeks = math.floor(date_dif/np.timedelta64(1,'W'))
        return weeks
    
    # apply function on each row in df
    preprocessed_df['week'] = preprocessed_df.apply(lambda row: difference_weeks(row['date'], row['release_date']), axis=1)
    
    if timeframe=='2weeks':
        preprocessed_df['2weeks'] = preprocessed_df['week'].apply(lambda week: week//2)
        
    # empty matrix to define nr of reviews per timeframe  
    reviews_per_timeframe = pd.DataFrame(np.zeros((len(df.groupby('game')), n_periods), dtype=np.int8), 
                                         index=df.game.unique())

    # enter reviews
    for index, row in preprocessed_df[preprocessed_df[timeframe]<n_periods].iterrows():
        reviews_per_timeframe.loc[row['game'], row[timeframe]] += 1

    reviews_per_timeframe.reset_index(inplace=True)
    reviews_per_timeframe.rename(columns={'index': 'game'}, inplace=True)
    
    # create panel dataset from matrix
    reviews_per_timeframe = reviews_per_timeframe.melt(id_vars=['game'], var_name=timeframe, value_name='nr_reviews')
    reviews_per_timeframe.sort_values(['game', timeframe], inplace=True)
    
    # calculate variables average grade and sentiment per timeframe
    reviews_grouped = preprocessed_df.groupby(['game', timeframe], as_index=False).agg({'grade': 'mean', 'sentiment': 'std'})
    reviews_grouped.rename(columns={'grade': 'user_avg_grade', 'sentiment': 'user_sentiment_var'}, inplace=True)
    
    # create panel dataset including average grade and sentiment per timeframe
    user_reviews = pd.merge(reviews_per_timeframe, reviews_grouped, how='left', on=['game', timeframe])
    
    # create a lag of time variant independent variables
    user_reviews_lagged = user_reviews
    user_reviews_lagged[['user_avg_grade', 'user_sentiment_var']] = user_reviews.groupby(['game'])['user_avg_grade', 'user_sentiment_var'].shift(1)

    return user_reviews_lagged

#### Assign user panel dataset
##### 2 weeks time frame

In [8]:
user_reviews_2weeks = user_preprocessing(users, '2weeks', 5)

In [9]:
user_reviews_2weeks.head()

,game,2weeks,nr_reviews,user_avg_grade,user_sentiment_var
0,007-legends-pc,0,7,NaN,NaN
1,007-legends-pc,1,1,1.428571,0.549131
2,007-legends-pc,2,0,100.000000,NaN
3,007-legends-pc,3,1,NaN,NaN
4,007-legends-pc,4,0,70.000000,NaN


##### One week time frame

In [10]:
user_reviews_week = user_preprocessing(users, 'week', 10)

In [11]:
user_reviews_week.head()

,game,week,nr_reviews,user_avg_grade,user_sentiment_var
0,007-legends-pc,0,6,NaN,NaN
1,007-legends-pc,1,1,0.0,0.578638
2,007-legends-pc,2,1,10.0,NaN
3,007-legends-pc,3,0,100.0,NaN
4,007-legends-pc,4,0,NaN,NaN


### Critic preprocessing

In [12]:
critics = pd.read_pickle('critic_review_clean.pkl')

In [13]:
def critic_preprocessing(df):
    # create aggregated variables for each game
    reviews_grouped = df.groupby(['game'], as_index=False).agg({'grade': 'mean', 'sentiment': 'std', 
                                                                        'review': 'count'})
    
    reviews_grouped.rename({'grade': 'critic_avg_grade', 'sentiment': 'critic_sentiment_var',
                       'review': 'critic_volume'}, inplace=True, axis=1)
    
    return reviews_grouped

#### Assign critics datasets

In [14]:
critic_reviews = critic_preprocessing(critics)

In [15]:
critic_reviews.head()

,game,critic_avg_grade,critic_sentiment_var,critic_volume
0,007-legends-pc,26.800000,0.351563,5
1,007-legends-playstation-3,42.318182,0.619410,22
2,007-legends-wii-u,40.666667,0.080168,3
3,007-legends-xbox-360,46.448980,0.669014,49
4,007-quantum-of-solace-ds,64.300000,0.411520,10


### Create final panel dataset
#### Panel df two weeks

In [16]:
panel_df_2weeks = pd.merge(user_reviews_2weeks, critic_reviews, how='left', on='game')

In [17]:
panel_df_2weeks.head()

,game,2weeks,nr_reviews,user_avg_grade,user_sentiment_var,critic_avg_grade,critic_sentiment_var,critic_volume
0,007-legends-pc,0,7,NaN,NaN,26.8,0.351563,5.0
1,007-legends-pc,1,1,1.428571,0.549131,26.8,0.351563,5.0
2,007-legends-pc,2,0,100.000000,NaN,26.8,0.351563,5.0
3,007-legends-pc,3,1,NaN,NaN,26.8,0.351563,5.0
4,007-legends-pc,4,0,70.000000,NaN,26.8,0.351563,5.0


#### Panel df two weeks enough reviews

In [18]:
# check minimum number of reviews over all periods for each game
panel_df_2weeks_grouped = panel_df_2weeks.groupby(['game'], as_index=False).agg({'nr_reviews': 'min'})
panel_df_2weeks_grouped.rename(columns={'nr_reviews': 'min_reviews'}, inplace=True)
panel_df_2weeks_grouped.head()

,game,min_reviews
0,007-legends-pc,0
1,007-legends-playstation-3,0
2,007-legends-wii-u,0
3,007-legends-xbox-360,0
4,007-quantum-of-solace-pc,0


In [19]:
# a list with games with more than 1 review in each timeframe
games_enough_reviews = list(panel_df_2weeks_grouped[panel_df_2weeks_grouped['min_reviews']>1]['game'])
len(games_enough_reviews)

483

In [20]:
# filter all games with more than 1 review in each timeframe
panel_df_2weeks_enough_reviews = panel_df_2weeks[panel_df_2weeks['game'].isin(games_enough_reviews)]

In [21]:
panel_df_2weeks_enough_reviews.head()

,game,2weeks,nr_reviews,user_avg_grade,user_sentiment_var,critic_avg_grade,critic_sentiment_var,critic_volume
55,1-2-switch-switch,0,36,NaN,NaN,58.190476,0.48737,65.0
56,1-2-switch-switch,1,12,63.888889,0.503946,58.190476,0.48737,65.0
57,1-2-switch-switch,2,2,53.333333,0.698784,58.190476,0.48737,65.0
58,1-2-switch-switch,3,2,0.000000,0.329582,58.190476,0.48737,65.0
59,1-2-switch-switch,4,2,55.000000,0.014991,58.190476,0.48737,65.0


#### Panel df week

In [22]:
panel_df_week = pd.merge(user_reviews_week, critic_reviews, how='left', on='game')

In [23]:
panel_df_week.head()

,game,week,nr_reviews,user_avg_grade,user_sentiment_var,critic_avg_grade,critic_sentiment_var,critic_volume
0,007-legends-pc,0,6,NaN,NaN,26.8,0.351563,5.0
1,007-legends-pc,1,1,0.0,0.578638,26.8,0.351563,5.0
2,007-legends-pc,2,1,10.0,NaN,26.8,0.351563,5.0
3,007-legends-pc,3,0,100.0,NaN,26.8,0.351563,5.0
4,007-legends-pc,4,0,NaN,NaN,26.8,0.351563,5.0


## Summary statistics
### 2 weeks timeframe

In [24]:
panel_df_2weeks.describe() # look especially at count, mean values and std

,nr_reviews,user_avg_grade,user_sentiment_var,critic_avg_grade,critic_sentiment_var,critic_volume
count,65340.000000,21916.000000,9391.000000,59275.000000,56765.000000,59300.000000
mean,1.256382,69.814345,0.516605,70.004170,0.477669,22.776644
std,4.290756,26.460174,0.369603,14.067308,0.138777,19.947632
min,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,0.000000,56.666667,0.131986,62.421053,0.404505,7.000000
50%,0.000000,78.888889,0.559013,72.354167,0.489651,17.000000
75%,1.000000,90.000000,0.800868,80.000000,0.562654,32.000000
max,191.000000,100.000000,1.409759,100.000000,1.310481,129.000000


### 2 weeks timeframe enough reviews

In [25]:
panel_df_2weeks_enough_reviews.describe()

,nr_reviews,user_avg_grade,user_sentiment_var,critic_avg_grade,critic_sentiment_var,critic_volume
count,2415.000000,1932.000000,1932.000000,2400.000000,2395.000000,2400.000000
mean,8.652174,66.433882,0.602992,79.783589,0.482034,46.945833
std,11.060424,21.833124,0.312272,9.638023,0.107202,25.513509
min,2.000000,0.000000,0.000354,32.958333,0.112320,1.000000
25%,3.000000,52.500000,0.369304,75.695135,0.407905,27.000000
50%,5.000000,70.323887,0.686604,81.594040,0.488107,43.000000
75%,9.000000,83.750000,0.814974,86.377589,0.564899,65.250000
max,176.000000,100.000000,1.392010,98.954545,0.786171,129.000000


### week timeframe

In [26]:
panel_df_week.describe()

,nr_reviews,user_avg_grade,user_sentiment_var,critic_avg_grade,critic_sentiment_var,critic_volume
count,130680.000000,30151.000000,11859.000000,118550.000000,113530.000000,118600.000000
mean,0.628191,69.356333,0.510027,70.004170,0.477669,22.776644
std,2.509015,27.454927,0.392297,14.067248,0.138776,19.947547
min,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,0.000000,55.000000,0.109298,62.421053,0.404505,7.000000
50%,0.000000,80.000000,0.516613,72.354167,0.489651,17.000000
75%,0.000000,90.000000,0.820215,80.000000,0.562654,32.000000
max,153.000000,100.000000,1.409759,100.000000,1.310481,129.000000


Remarks:
* mean nr_reviews: very low --> data sparsity
* count of user_sentiment_var: very low compared to entire dataset

### Summary statistics per period

In [27]:
def sum_stats_per_period(df, timeframe):
    mean = df.groupby(timeframe).mean().transpose()
    std = df.groupby(timeframe).std().transpose()
    
    mean.rename(columns = {0: 'mean_0',
                      1: 'mean_1',
                      2: 'mean_2',
                      3: 'mean_3',
                      4: 'mean_4',
                      5: 'mean_5',
                      6: 'mean_6',
                      7: 'mean_7',
                      8: 'mean_8',
                      9: 'mean_9'},
                       inplace=True)
    
    std.rename(columns = {0: 'std_0',
                      1: 'std_1',
                      2: 'std_2',
                      3: 'std_3',
                      4: 'std_4',
                      5: 'std_5',
                      6: 'std_6',
                      7: 'std_7',
                      8: 'std_8',
                      9: 'std_9'},
                       inplace=True)
    return mean.join(std)

#### 2 weeks timeframe

In [28]:
sum_stats_per_period(panel_df_2weeks, '2weeks')

2weeks,mean_0,mean_1,mean_2,mean_3,mean_4,std_0,std_1,std_2,std_3,std_4
nr_reviews,4.003367,0.868993,0.554484,0.465794,0.389272,8.322542,2.641796,1.541552,1.573429,1.181694
user_avg_grade,NaN,71.083674,68.288503,67.082653,68.426182,NaN,26.111579,25.946110,27.472371,27.340277
user_sentiment_var,NaN,0.501503,0.531683,0.548002,0.510676,NaN,0.349388,0.375434,0.392724,0.400712
critic_avg_grade,70.004170,70.004170,70.004170,70.004170,70.004170,14.067783,14.067783,14.067783,14.067783,14.067783
critic_sentiment_var,0.477669,0.477669,0.477669,0.477669,0.477669,0.138782,0.138782,0.138782,0.138782,0.138782
critic_volume,22.776644,22.776644,22.776644,22.776644,22.776644,19.948304,19.948304,19.948304,19.948304,19.948304


#### 2 weeks timeframe enough reviews

In [29]:
sum_stats_per_period(panel_df_2weeks_enough_reviews, '2weeks')

2weeks,mean_0,mean_1,mean_2,mean_3,mean_4,std_0,std_1,std_2,std_3,std_4
nr_reviews,21.322981,7.720497,5.308489,4.790890,4.118012,17.415690,7.884181,3.175710,4.333571,3.138660
user_avg_grade,NaN,69.750806,65.031285,64.725465,66.227970,NaN,20.237449,21.647385,22.049276,23.008924
user_sentiment_var,NaN,0.605137,0.626807,0.617374,0.562651,NaN,0.236221,0.293911,0.336697,0.364284
critic_avg_grade,79.783589,79.783589,79.783589,79.783589,79.783589,9.646069,9.646069,9.646069,9.646069,9.646069
critic_sentiment_var,0.482034,0.482034,0.482034,0.482034,0.482034,0.107292,0.107292,0.107292,0.107292,0.107292
critic_volume,46.945833,46.945833,46.945833,46.945833,46.945833,25.534805,25.534805,25.534805,25.534805,25.534805


#### week timeframe

In [30]:
sum_stats_per_period(panel_df_week, 'week')

week,mean_0,mean_1,mean_2,mean_3,mean_4,mean_5,mean_6,mean_7,mean_8,mean_9,std_0,std_1,std_2,std_3,std_4,std_5,std_6,std_7,std_8,std_9
nr_reviews,3.179369,0.823998,0.495103,0.373890,0.299357,0.255127,0.240282,0.225513,0.197735,0.191537,6.426677,2.439773,1.525846,1.327954,0.916208,0.858176,0.913723,0.851597,0.683753,0.663562
user_avg_grade,NaN,71.299583,69.365053,68.165507,67.692786,66.638561,66.671392,69.062374,67.228587,66.994494,NaN,26.546406,25.901336,26.856206,28.326701,29.203373,29.087317,28.524324,29.009069,29.164989
user_sentiment_var,NaN,0.491029,0.519194,0.530807,0.516204,0.518394,0.548052,0.484385,0.532323,0.495593,NaN,0.360374,0.375526,0.397460,0.411900,0.422574,0.426936,0.418220,0.434612,0.432362
critic_avg_grade,70.004170,70.004170,70.004170,70.004170,70.004170,70.004170,70.004170,70.004170,70.004170,70.004170,14.067783,14.067783,14.067783,14.067783,14.067783,14.067783,14.067783,14.067783,14.067783,14.067783
critic_sentiment_var,0.477669,0.477669,0.477669,0.477669,0.477669,0.477669,0.477669,0.477669,0.477669,0.477669,0.138782,0.138782,0.138782,0.138782,0.138782,0.138782,0.138782,0.138782,0.138782,0.138782
critic_volume,22.776644,22.776644,22.776644,22.776644,22.776644,22.776644,22.776644,22.776644,22.776644,22.776644,19.948304,19.948304,19.948304,19.948304,19.948304,19.948304,19.948304,19.948304,19.948304,19.948304


Remarks:
* mean nr_reviews: very low, especially after period 1 --> data sparsity
* less sparsity for 2 weeks period with only games with enough reviews in each period, however probably still too sparse for reliable variances

## Correlations

### 2 weeks timeframe

In [31]:
panel_df_2weeks.corr()

,nr_reviews,user_avg_grade,user_sentiment_var,critic_avg_grade,critic_sentiment_var,critic_volume
nr_reviews,1.000000,-0.040329,0.118880,0.091909,0.022174,0.215805
user_avg_grade,-0.040329,1.000000,-0.380810,0.326457,-0.056829,0.093584
user_sentiment_var,0.118880,-0.380810,1.000000,-0.096712,0.125443,0.020012
critic_avg_grade,0.091909,0.326457,-0.096712,1.000000,-0.134207,0.358971
critic_sentiment_var,0.022174,-0.056829,0.125443,-0.134207,1.000000,0.071815
critic_volume,0.215805,0.093584,0.020012,0.358971,0.071815,1.000000


### 2 weeks timeframe enough reviews

In [32]:
panel_df_2weeks_enough_reviews.corr()

,nr_reviews,user_avg_grade,user_sentiment_var,critic_avg_grade,critic_sentiment_var,critic_volume
nr_reviews,1.000000,-0.023055,0.037247,-0.060990,0.059812,0.072220
user_avg_grade,-0.023055,1.000000,-0.381537,0.318880,-0.057794,0.246644
user_sentiment_var,0.037247,-0.381537,1.000000,-0.107901,0.129541,-0.131933
critic_avg_grade,-0.060990,0.318880,-0.107901,1.000000,-0.418916,0.144385
critic_sentiment_var,0.059812,-0.057794,0.129541,-0.418916,1.000000,-0.036288
critic_volume,0.072220,0.246644,-0.131933,0.144385,-0.036288,1.000000


### week timeframe

In [33]:
panel_df_week.corr()

,nr_reviews,user_avg_grade,user_sentiment_var,critic_avg_grade,critic_sentiment_var,critic_volume
nr_reviews,1.000000,-0.035275,0.097199,0.078586,0.018960,0.184523
user_avg_grade,-0.035275,1.000000,-0.358016,0.293598,-0.056525,0.087698
user_sentiment_var,0.097199,-0.358016,1.000000,-0.097016,0.116881,-0.011637
critic_avg_grade,0.078586,0.293598,-0.097016,1.000000,-0.134207,0.358971
critic_sentiment_var,0.018960,-0.056525,0.116881,-0.134207,1.000000,0.071815
critic_volume,0.184523,0.087698,-0.011637,0.358971,0.071815,1.000000


## Panel Regression Analysis

In [34]:
def panel_regression(df, timeframe):
    '''
    Run a panel regression with time fixed effects
    '''
    data = df.set_index(['game', timeframe])
    mod = PanelOLS.from_formula('''nr_reviews ~ 1 + user_sentiment_var*critic_sentiment_var + user_avg_grade + critic_avg_grade + 
                                critic_volume + TimeEffects''', data, )
    res = mod.fit(cov_type='unadjusted')
    return res

### Panel Regression with time frames of two weeks

In [35]:
panel_regression(panel_df_2weeks, '2weeks')

/Users/myrthe/anaconda3/lib/python3.7/site-packages/linearmodels/utility.py:478: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


Dep. Variable:,nr_reviews,R-squared:,0.0969
Estimator:,PanelOLS,R-squared (Between):,0.0993
No. Observations:,9049,R-squared (Within):,-0.0252
Date:,"Sat, Oct 26 2019",R-squared (Overall):,0.0922
Time:,11:13:20,Log-likelihood,-2.392e+04
Cov. Estimator:,Unadjusted,,
,,F-statistic:,161.69
Entities:,4576,P-value,0.0000
Avg Obs:,1.9775,Distribution:,"F(6,9039)"
Min Obs:,1.0000,,
Max Obs:,4.0000,F-statistic (robust):,161.69


**Remarks**
* A lot of instances were not included in the regression, because of missing values (due to 0 or 1 reviews within time period)
* Three significant control variables at 1% level:
    * user_avg_grade: the higher user_avg_grade, the lower nr_reviews (proxy for performance)
        --> This is not what one would expect, as we would expect that a higher user_avg_grade improves performance. However, it could be that with our proxy for performance (nr_reviews), users leave more reviews when they are extremely dissatisfied with a product.
    * critic_avg_grade: the higher critic_avg_grade, the higher nr_reviews (proxy for perfomance)
        --> In line with expectations
    * critic_volume: the higher critic_volume, the higher nr_reviews (proxy for perfomance)
        --> In line with expectations
* Independent variable is significant at 10% level:
    The higher user_sentiment_var, the higher nr_reviews (proxy for performance) --> contrary to what was expected. However, we have to sparse data in order to calculate reliable sentiment variance variables. 
* Moderator is not significant

### Panel regression with time frames of two weeks and only games with enough reviews

In [36]:
panel_regression(panel_df_2weeks_enough_reviews, '2weeks')

Dep. Variable:,nr_reviews,R-squared:,0.0275
Estimator:,PanelOLS,R-squared (Between):,0.0481
No. Observations:,1916,R-squared (Within):,-0.0102
Date:,"Sat, Oct 26 2019",R-squared (Overall):,0.0225
Time:,11:13:20,Log-likelihood,-5788.6
Cov. Estimator:,Unadjusted,,
,,F-statistic:,8.9725
Entities:,479,P-value,0.0000
Avg Obs:,4.0000,Distribution:,"F(6,1906)"
Min Obs:,4.0000,,
Max Obs:,4.0000,F-statistic (robust):,8.9725


**Remarks**
* A lot of instances were not included in the regression, because of missing values (due to 0 or 1 reviews within time period)
* Two significant control variables at 1% level:
    * user_avg_grade: the higher user_avg_grade, the lower nr_reviews (proxy for performance)
        --> This is not what one would expect, as we would expect that a higher user_avg_grade improves performance. However, it could be that with our proxy for performance (nr_reviews), users leave more reviews when they are extremely dissatisfied with a product.
    * critic_volume: the higher critic_volume, the higher nr_reviews (proxy for perfomance)
        --> In line with expectations
* Independent variable is not significant
* Moderator is not significant

### Panel Regression with time frames of one week

In [37]:
panel_regression(panel_df_week, 'week')

Dep. Variable:,nr_reviews,R-squared:,0.0749
Estimator:,PanelOLS,R-squared (Between):,0.0047
No. Observations:,11522,R-squared (Within):,-0.0191
Date:,"Sat, Oct 26 2019",R-squared (Overall):,0.0487
Time:,11:13:21,Log-likelihood,-2.889e+04
Cov. Estimator:,Unadjusted,,
,,F-statistic:,155.19
Entities:,4213,P-value,0.0000
Avg Obs:,2.7349,Distribution:,"F(6,11507)"
Min Obs:,1.0000,,
Max Obs:,9.0000,F-statistic (robust):,155.19


**Remarks**
* A lot of instances were not included in the regression, because of missing values (due to 0 or 1 reviews within time period)
* Three significant control variables at 1% level:
    * user_avg_grade: the higher user_avg_grade, the lower nr_reviews (proxy for performance)
        --> This is not what one would expect, as we would expect that a higher user_avg_grade improves performance. However, it could be that with our proxy for performance (nr_reviews), users leave more reviews when they are extremely dissatisfied with a product.
    * critic_avg_grade: the higher critic_avg_grade, the higher nr_reviews (proxy for perfomance)
        --> In line with expectations
    * critic_volume: the higher critic_volume, the higher nr_reviews (proxy for perfomance)
        --> In line with expectations
* Independent variable is not significant
* Moderator is not significant